In [1]:
import numpy as np

mean = np.array([0.0, 1.0])
cov = np.array([[1.0, 0.5], [0.5, 1.0]])
size = 100_000

# Method 1: NumPy default_rng().multivariate_normal
rng = np.random.default_rng(seed=42)
samples_mv = rng.multivariate_normal(mean, cov, size=size)

# Method 2: Manual sampling
rng = np.random.default_rng(seed=42)
eps = rng.standard_normal(size=(size, 2))
L = np.linalg.cholesky(cov)
samples_manual = mean + eps @ L.T

# Compare sample mean and covariance
print("Multivariate Normal (mean):", samples_mv.mean(axis=0))
print("Manual Sampling (mean):     ", samples_manual.mean(axis=0))

print("Multivariate Normal (cov):\n", np.cov(samples_mv, rowvar=False))
print("Manual Sampling (cov):\n", np.cov(samples_manual, rowvar=False))

Multivariate Normal (mean): [0.00156743 1.00226364]
Manual Sampling (mean):      [-0.00221187  0.999497  ]
Multivariate Normal (cov):
 [[0.99977376 0.4947176 ]
 [0.4947176  1.00322467]]
Manual Sampling (cov):
 [[0.99747788 0.49701348]
 [0.49701348 1.00781644]]


In [2]:
print(np.allclose(samples_mv.mean(axis=0), mean, atol=1e-2))
print(np.allclose(np.cov(samples_mv, rowvar=False), cov, atol=1e-2))

True
True


In [3]:
from typing import Union, Iterable, List, Tuple

class _NumpyRNG:
    
    def __init__(self, seed):
        self.seed = seed
        self.rng = np.random.default_rng(self.seed)

    def rand(self, size=None):
        return self.rng.random(size)

    def randint(self, low: int, high: int = None, size: int = None):
        return self.rng.integers(low=low, high=high, size=size)

    def choice(self, a: Union[int, Iterable[int]], size: Union[int, Tuple[int]] = None, p: Iterable[float] = None):
        return self.rng.choice(a=a, size=size, p=p)

    def beta(self, num_success: int, num_failure: int, size=None):
        return self.rng.beta(num_success, num_failure, size)

    def standard_normal(self, size=None):
        return self.rng.standard_normal(size)

    def multivariate_normal(self, mean: Union[np.ndarray, List[float]],
                            covariance: Union[np.ndarray, List[List[float]]], size=None):
        return np.squeeze(self.rng.multivariate_normal(mean, covariance, size=size, method='cholesky'))

    def dirichlet(self, alpha: List[float], size=None):
        return self.rng.dirichlet(alpha, size)

In [4]:
import numpy as np

class _RidgeRegression:

    def __init__(self, rng, alpha = 1.0, l2_lambda = 1.0, scale: bool = False):

        # Ridge Regression: https://onlinecourses.science.psu.edu/stat857/node/155/
        self.rng = rng                      # random number generator
        self.alpha = alpha                  # exploration parameter
        self.l2_lambda = l2_lambda          # regularization parameter
        self.scale = scale                  # scale contexts

        self.beta = None                    # (XtX + l2_lambda * I_d)^-1 * Xty = A^-1 * Xty
        self.A = None                       # (XtX + l2_lambda * I_d)
        self.A_inv = None                   # (XtX + l2_lambda * I_d)^-1
        self.Xty = None
        self.scaler = None

    def init(self, num_features: int):
        # By default, assume that
        # A is the identity matrix and Xty is set to 0
        self.Xty = np.zeros(num_features)
        self.A = self.l2_lambda * np.identity(num_features)
        self.A_inv = self.A.copy()
        self.beta = np.dot(self.A_inv, self.Xty)

    def fit(self, X: np.ndarray, y: np.ndarray):

        # Scale
        if self.scaler is not None:
            X = X.astype('float64')
            if not hasattr(self.scaler, 'scale_'):
                self.scaler.fit(X)
            else:
                self.scaler.partial_fit(X)
            X = self.scaler.transform(X)

        # X transpose
        Xt = X.T

        # Update A
        self.A = self.A + np.dot(Xt, X)
        self.A_inv = np.linalg.inv(self.A)

        # Add new Xty values to old
        self.Xty = self.Xty + np.dot(Xt, y)

        # Recalculate beta coefficients
        self.beta = np.dot(self.A_inv, self.Xty)

    def predict(self, x: np.ndarray):

        # Scale
        if self.scaler is not None:
            x = self._scale_predict_context(x)

        # Calculate default expectation y = x * b
        return np.dot(x, self.beta)

    def _scale_predict_context(self, x: np.ndarray):
        if not hasattr(self.scaler, 'scale_'):
            return x

        # Transform and return to previous shape. Convert to float64 to suppress any type warnings.
        return self.scaler.transform(x.astype('float64'))


class _LinTS(_RidgeRegression):

    def predict(self, x: np.ndarray):
        # Scale
        if self.scaler is not None:
            x = self._scale_predict_context(x)

        # Randomly sample coefficients from multivariate normal distribution
        # Covariance is enhanced with the exploration factor
        # Generates  random samples for all contexts in one single go. type(beta_sampled): np.ndarray
        beta_sampled = self.rng.multivariate_normal(self.beta, np.square(self.alpha) * self.A_inv, size=x.shape[0])
        
        print(beta_sampled[0])

        # Calculate expectation y = x * beta_sampled
        return np.sum(x * beta_sampled, axis=1)

In [5]:
class _LinTS_modified(_RidgeRegression):

    def predict(self, x: np.ndarray):
        # Scale
        if self.scaler is not None:
            x = self._scale_predict_context(x)

        # Randomly sample coefficients from multivariate normal distribution
        # Covariance is enhanced with the exploration factor
        # Generates  random samples for all contexts in one single go. type(beta_sampled): np.ndarray
        eps = rng.standard_normal(size=(x.shape[0], self.beta.shape[0]))
        L = np.linalg.cholesky(np.square(self.alpha) * self.A_inv)
        mean = self.beta
        beta_sampled = mean + eps @ L.T

        print(self.beta)

        # Calculate expectation y = x * beta_sampled
        return np.sum(x * beta_sampled, axis=1)

In [6]:
TEST_INTERACTIONS_SIZE = 10_000_000

In [7]:
X_train = np.array([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0]])
y_train = np.array([1, 0])

X_test = np.array([[0, 1, 0, 0, 0], [1, 0, 0, 0, 0]])
num_contexts = X_test.shape[0]
X_test = np.repeat(X_test, TEST_INTERACTIONS_SIZE, axis=0)

In [8]:
lin_ts1 = _LinTS(rng=_NumpyRNG(seed=42), alpha=1.0, l2_lambda=1.0, scale=False)
lin_ts1.init(num_features=5)


lin_ts1.fit(X_train, y_train)

scores1 = lin_ts1.predict(X_test).reshape((num_contexts, TEST_INTERACTIONS_SIZE)).T

[ 0.21546751 -0.23537981  0.7504512   0.94056472 -1.95103519]


In [9]:
scores1

array([[-0.23537981, -0.12584549],
       [ 0.59039682, -1.31946612],
       [ 1.04998195, -0.39989925],
       ...,
       [-0.1319411 ,  0.3742013 ],
       [ 0.88785896,  0.85114717],
       [ 0.50719944, -0.48063021]])

In [10]:
lin_ts2 = _LinTS_modified(rng=np.random.default_rng(42), alpha=1.0, l2_lambda=1.0, scale=False)
lin_ts2.init(num_features=5)

lin_ts2.fit(X_train, y_train)

lin_ts2.predict(X_test)

scores2 = lin_ts2.predict(X_test).reshape((num_contexts, TEST_INTERACTIONS_SIZE)).T

[0.  0.5 0.  0.  0. ]
[0.  0.5 0.  0.  0. ]


In [11]:

print('LinTS original mean: ', scores1.mean(axis=0))
print('LinTS modified mean: ', scores2.mean(axis=0))

# Check if the means are close enough
print(np.allclose(scores1.mean(axis=0), scores2.mean(axis=0), atol=1e-2))
print(np.allclose(scores1.mean(axis=0), scores2.mean(axis=0), atol=1e-2))

print("LinTS covariance: ", np.cov(scores1, rowvar=False))
print("LinTS modified covariance: ", np.cov(scores2, rowvar=False))
print(np.allclose(np.cov(scores1, rowvar=False), np.cov(scores2, rowvar=False), atol=1e-2))
print(np.allclose(np.cov(scores1, rowvar=False), np.cov(scores2, rowvar=False), atol=1e-2))

LinTS original mean:  [ 4.99974308e-01 -8.29050524e-06]
LinTS modified mean:  [4.99541534e-01 7.57056431e-05]
True
True
LinTS covariance:  [[5.00187258e-01 9.74367480e-05]
 [9.74367480e-05 5.00391862e-01]]
LinTS modified covariance:  [[4.99720409e-01 2.45057695e-04]
 [2.45057695e-04 4.99938372e-01]]
True
True


In [12]:
import torch

class _LinTS_modified2(_RidgeRegression):

    def predict(self, x: np.ndarray):
        # Scale
        if self.scaler is not None:
            x = self._scale_predict_context(x)

        # Randomly sample coefficients from multivariate normal distribution
        # Covariance is enhanced with the exploration factor
        # Generates  random samples for all contexts in one single go. type(beta_sampled): np.ndarray
        eps = torch.from_numpy(self.rng.standard_normal(size=(x.shape[0], self.beta.shape[0]))).cuda()
        L = torch.linalg.cholesky(torch.from_numpy(np.square(self.alpha) * self.A_inv).cuda())
        mean = torch.from_numpy(self.beta).cuda()
        beta_sampled = mean + eps @ L

        # Calculate expectation y = x * beta_sampled
        return torch.sum(torch.from_numpy(x).cuda() * beta_sampled, axis=1)

In [13]:
lin_ts3 = _LinTS_modified2(rng=np.random.default_rng(42), alpha=1.0, l2_lambda=1.0, scale=False)
lin_ts3.init(num_features=5)

lin_ts3.fit(X_train, y_train)

scores3 = lin_ts3.predict(X_test).reshape((num_contexts, TEST_INTERACTIONS_SIZE)).T.cpu().numpy()

In [14]:

print('LinTS original mean: ', scores1.mean(axis=0))
print('LinTS modified mean: ', scores3.mean(axis=0))

# Check if the means are close enough
np.allclose(scores1.mean(axis=0), scores3.mean(axis=0), atol=1e-2)
np.allclose(scores1.mean(axis=0), scores3.mean(axis=0), atol=1e-2)

print("LinTS covariance: ", np.cov(scores1, rowvar=False))
print("LinTS modified covariance: ", np.cov(scores3, rowvar=False))
np.allclose(np.cov(scores1, rowvar=False), np.cov(scores3, rowvar=False), atol=1e-2)
np.allclose(np.cov(scores1, rowvar=False), np.cov(scores3, rowvar=False), atol=1e-2)

LinTS original mean:  [ 4.99974308e-01 -8.29050524e-06]
LinTS modified mean:  [ 4.99974308e-01 -8.29050524e-06]
LinTS covariance:  [[5.00187258e-01 9.74367480e-05]
 [9.74367480e-05 5.00391862e-01]]
LinTS modified covariance:  [[5.00187258e-01 9.74367480e-05]
 [9.74367480e-05 5.00391862e-01]]


True

## Testes com mais de um item

In [15]:
from copy import deepcopy
from typing import Callable, Dict, List, Optional, Union

import numpy as np

from mabwiser.base_mab import BaseMAB
from mabwiser.utils import Arm, Num, _BaseRNG

class _Linear(BaseMAB):
    factory = {"ts": _LinTS,  "ridge": _RidgeRegression}

    def __init__(self, rng: _BaseRNG, arms: List[Arm], n_jobs: int, backend: Optional[str],
                 alpha: Num, epsilon: Num, l2_lambda: Num, regression: str, scale: bool):
        super().__init__(rng, arms, n_jobs, backend)
        self.alpha = alpha
        self.epsilon = epsilon
        self.l2_lambda = l2_lambda
        self.regression = regression
        self.scale = scale
        self.num_features = None

        # Create regression model for each arm
        self.arm_to_model = dict((arm, _Linear.factory.get(regression)(rng, alpha, l2_lambda, scale)) for arm in arms)

    def fit(self, decisions: np.ndarray, rewards: np.ndarray, contexts: np.ndarray = None) -> None:

        # Initialize each model by arm
        self.num_features = contexts.shape[1]
        for arm in self.arms:
            self.arm_to_model[arm].init(num_features=self.num_features)

        # Reset warm started arms
        self._reset_arm_to_status()

        # Perform parallel fit
        self._parallel_fit(decisions, rewards, contexts)

        # Update trained arms
        self._set_arms_as_trained(decisions=decisions, is_partial=False)

    def partial_fit(self, decisions: np.ndarray, rewards: np.ndarray, contexts: np.ndarray = None) -> None:
        # Perform parallel fit
        self._parallel_fit(decisions, rewards, contexts)

        # Update trained arms
        self._set_arms_as_trained(decisions=decisions, is_partial=True)

    def predict(self, contexts: np.ndarray = None) -> Union[Arm, List[Arm]]:
        # Return predict for the given context
        return self._vectorized_predict_context(contexts, is_predict=True)

    def predict_expectations(self, contexts: np.ndarray = None) -> Union[Dict[Arm, Num], List[Dict[Arm, Num]]]:
        # Return predict expectations for the given context
        return self._vectorized_predict_context(contexts, is_predict=False)

    def warm_start(self, arm_to_features: Dict[Arm, List[Num]], distance_quantile: float):
        self._warm_start(arm_to_features, distance_quantile)

    def _copy_arms(self, cold_arm_to_warm_arm):
        for cold_arm, warm_arm in cold_arm_to_warm_arm.items():
            self.arm_to_model[cold_arm] = deepcopy(self.arm_to_model[warm_arm])

    def _uptake_new_arm(self, arm: Arm, binarizer: Callable = None):

        # Add to untrained_arms arms
        self.arm_to_model[arm] = _Linear.factory.get(self.regression)(self.rng, self.alpha, self.l2_lambda, self.scale)

        # If fit happened, initialize the new arm to defaults
        is_fitted = self.num_features is not None
        if is_fitted:
            self.arm_to_model[arm].init(num_features=self.num_features)

    def _fit_arm(self, arm: Arm, decisions: np.ndarray, rewards: np.ndarray, contexts: Optional[np.ndarray] = None):

        # Get local copy of model to minimize communication overhead
        # between arms (processes) using shared object
        lr = deepcopy(self.arm_to_model[arm])

        # Skip the arms with no data
        indices = np.where(decisions == arm)
        if indices[0].size == 0:
            return lr

        # Fit the regression
        X = contexts[indices]
        y = rewards[indices]
        lr.fit(X, y)

        self.arm_to_model[arm] = lr

    def _predict_contexts(self, contexts: np.ndarray, is_predict: bool,
                          seeds: Optional[np.ndarray] = None, start_index: Optional[int] = None) -> List:
        pass

    def _vectorized_predict_context(self, contexts: np.ndarray, is_predict: bool) -> List:

        # Converting the arms list to numpy array
        arms = deepcopy(self.arms)
        arms = np.array(arms)

        # Initializing array with expectations for each arm
        num_contexts = contexts.shape[0]
        arm_expectations = np.empty((num_contexts, len(arms)), dtype=float)

        # With epsilon probability, assign random flag to context
        random_values = self.rng.rand(num_contexts)
        random_mask = np.array(random_values < self.epsilon)
        random_indices = random_mask.nonzero()[0]

        # For random indices, generate random expectations
        arm_expectations[random_indices] = self.rng.rand((random_indices.shape[0], len(arms)))

        # For non-random indices, get expectations for each arm
        nonrandom_indices = np.where(~random_mask)[0]
        nonrandom_context = contexts[nonrandom_indices]
        arm_expectations[nonrandom_indices] = np.array([self.arm_to_model[arm].predict(nonrandom_context)
                                                        for arm in arms]).T

        if is_predict:
            predictions = arms[np.argmax(arm_expectations, axis=1)].tolist()
        else:
            predictions = [dict(zip(self.arms, value)) for value in arm_expectations]

        return predictions if len(predictions) > 1 else predictions[0]

    def _drop_existing_arm(self, arm: Arm) -> None:
        self.arm_to_model.pop(arm)

In [16]:
arms = [0, 1, 2, 3]
lints_more_arms_1 = _Linear(
    rng=_NumpyRNG(seed=42),
    arms=arms,
    n_jobs=1,
    alpha=1.0,
    epsilon=0.0,
    backend=None,
    l2_lambda=1.0,
    regression='ts',
    scale=False
)



In [17]:
X_train = np.array([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0],
                    [0, 0, 1, 0, 0], [0, 0, 0, 1, 0],
                    [0, 0, 0, 0, 1], [0, 0, 1, 0, 0],
                    [0, 0, 1, 0, 0], [0, 0, 0, 0, 1]], dtype=float)
decisions = np.array([0, 0, 1, 1, 2, 2, 3, 3])
rewards = np.array([1, 0, 1, 0, 1, 0, 1, 0])

X_test = np.array([[0, 0, 1, 0, 0]])
num_contexts = X_test.shape[0]
X_test = np.repeat(X_test, TEST_INTERACTIONS_SIZE, axis=0)

In [18]:
lints_more_arms_1.fit(decisions, rewards, contexts=X_train)

In [19]:
scores_more_1_dict = lints_more_arms_1.predict_expectations(X_test)

scores_more_1 = np.empty((TEST_INTERACTIONS_SIZE, len(arms)), dtype=np.double)
for arm in arms:
    for i in range(TEST_INTERACTIONS_SIZE):
        scores_more_1[i, arm] = scores_more_1_dict[i][arm]

[ 0.71546751 -0.73537981  0.7504512   0.94056472 -1.95103519]
[ 0.30471708 -1.03998411  1.03064913  0.66507969 -1.95103519]
[ 0.30471708 -1.03998411  0.53064913  0.94056472 -0.87959021]
[ 0.30471708 -1.03998411  1.03064913  0.94056472 -1.37959021]


In [20]:

print('LinTS original mean: ', scores_more_1.mean(axis=0))

print("LinTS covariance: ", np.cov(scores_more_1, rowvar=False))

LinTS original mean:  [1.56298524e-04 5.00110520e-01 1.10519746e-04 5.00110520e-01]
LinTS covariance:  [[1.00002563 0.70712491 0.70712491 0.70712491]
 [0.70712491 0.50001282 0.50001282 0.50001282]
 [0.70712491 0.50001282 0.50001282 0.50001282]
 [0.70712491 0.50001282 0.50001282 0.50001282]]


In [21]:
# Code adapted from: https://github.com/fidelity/mabwiser/blob/master/mabwiser/linear.py
# The difference is that the original code accept different types of arms (strings, integers, etc) and the modified code only accept sequential integers as arms. With this modification, the code can be optimized.

ITEMS_PER_BATCH = 10_000
INTERACTIONS_PER_BATCH_LINTS = 1_000

from mabwiser.linear import _Linear
from mabwiser.utils import Arm, Num, _BaseRNG
from typing import Callable, Dict, List, Optional, Union
import numpy as np
import time
import torch

class _RidgeRegressionOptimized:

    def __init__(self, rng: _BaseRNG, alpha: Num = 1.0, l2_lambda: Num = 1.0, scale: bool = False):

        # Ridge Regression: https://onlinecourses.science.psu.edu/stat857/node/155/
        self.rng = rng                      # random number generator
        self.alpha = alpha                  # exploration parameter
        self.l2_lambda = l2_lambda          # regularization parameter
        self.scale = scale                  # scale contexts

        self.beta = None                    # (XtX + l2_lambda * I_d)^-1 * Xty = A^-1 * Xty
        self.A = None                       # (XtX + l2_lambda * I_d)
        self.A_inv = None                   # (XtX + l2_lambda * I_d)^-1
        self.Xty = None
        self.scaler = None

    def init(self, num_features: int, num_arms: int):
        # By default, assume that
        # A is the identity matrix and Xty is set to 0
        start_time = time.time()
        self.Xty = torch.zeros((num_arms, num_features), device='cuda', dtype=torch.double)
        self.A = torch.eye(num_features, device='cuda', dtype=torch.double).unsqueeze(0).repeat(num_arms, 1, 1) * self.l2_lambda
        #self.A_inv = self.A.clone()
        self.beta = torch.zeros((num_arms, num_features), device='cuda', dtype=torch.double)
        print(f'init demorou {time.time() - start_time} segundos')
        #self.scaler = StandardScaler() if self.scale else None

    def fit(self, decisions: np.ndarray, X: np.ndarray, y: np.ndarray):

        # Scale
        #if self.scaler is not None:
        #    X = X.astype('float64')
        #    if not hasattr(self.scaler, 'scale_'):
        #        self.scaler.fit(X)
        #    else:
        #        self.scaler.partial_fit(X)
        #    fix_small_variance(self.scaler)
        #    X = self.scaler.transform(X)
        start_time = time.time()
        X_device = torch.tensor(X, device='cuda')
        y_device = torch.tensor(y, device='cuda')
        decisions_device = torch.tensor(decisions, device='cuda')
        print(f'passar para cuda demorou {time.time() - start_time} segundos')
        # Update A
        #start_time = time.time()
        #outer = torch.einsum('ni,nj->nij', X_device, X_device)  # (n, d, d)
        #print(f'outer demorou {time.time() - start_time} segundos')

        # Scatter add outer products into self.A based on decisions
        start_time = time.time()
        self.A.index_add_(0, decisions_device, torch.einsum('ni,nj->nij', X_device, X_device))
        print(f'A add demorou {time.time() - start_time} segundos')

        # Add X * y to Xty
        start_time = time.time()
        self.Xty.index_add_(0, decisions_device, X_device * y_device.view(-1, 1))
        print(f'Xty demorou {time.time() - start_time} segundos')

        # Invert each A matrix
        for j in range(0, self.beta.shape[0], ITEMS_PER_BATCH):            
            start_time = time.time()
            self.beta[j:j+ITEMS_PER_BATCH] = torch.linalg.solve(
                self.A[j:j+ITEMS_PER_BATCH],
                self.Xty[j:j+ITEMS_PER_BATCH]
            )
            print(f'beta demorou {time.time() - start_time} segundos')

    def predict(self, x: np.ndarray):

        # Calculate default expectation y = x * b
        return torch.matmul(torch.tensor(x, device='cuda', dtype=torch.double), self.beta.T)

class _LinTSOptimized(_RidgeRegressionOptimized):

    def __init__(self, rng: _BaseRNG, alpha: Num = 1.0, l2_lambda: Num = 1.0, scale: bool = False):
        super().__init__(rng, alpha, l2_lambda, scale)
        self.torch_rng = torch.Generator(device='cuda').manual_seed(42)
    
    def predict(self, x: np.ndarray):
        x_torch = torch.tensor(x, device='cuda', dtype=torch.double)  # [B, D]

        num_arms, d = self.beta.shape
        B = x.shape[0]

        scores = torch.empty((B, num_arms), device='cuda', dtype=torch.double)

        # z_all = torch.randn((num_arms, B, d), generator=self.torch_rng, device='cuda', dtype=torch.double)

        for start in range(0, num_arms, ITEMS_PER_BATCH):
            end = min(start + ITEMS_PER_BATCH, num_arms)
            chunk_size = end - start

            beta_chunk = self.beta[start:end]            # [chunk_size, D]
            A_chunk = self.A[start:end]                  # [chunk_size, D, D]

            # Cholesky decomposition
            L_chunk = torch.linalg.cholesky((self.alpha ** 2) * A_chunk)  # [chunk_size, D, D]

            z = torch.randn((chunk_size, B, d), generator=self.torch_rng, device='cuda', dtype=torch.double)

            # beta_sampled: [chunk_size, B, D]
            beta_sampled = beta_chunk[:, None, :] + torch.matmul(L_chunk, z.transpose(1, 2)).transpose(1, 2)

            # Compute scores for this chunk: [B, chunk_size]
            scores_chunk = torch.einsum('mbd,bd->bm', beta_sampled, x_torch)

            # Assign to output
            scores[:, start:end] = scores_chunk

        return scores  # shape: [B, M]

In [22]:
lints_more_arms_2 = _LinTSOptimized(
    rng=_NumpyRNG(seed=42),
    alpha=1.0,
    l2_lambda=1.0,
    scale=False
)

In [23]:
lints_more_arms_2.init(num_features=5, num_arms=len(arms))

lints_more_arms_2.fit(decisions, X_train, rewards)

init demorou 0.029323577880859375 segundos
passar para cuda demorou 0.0004322528839111328 segundos
A add demorou 0.0965576171875 segundos
Xty demorou 0.0003063678741455078 segundos
beta demorou 0.024651527404785156 segundos


In [24]:
scores_more_2 = lints_more_arms_2.predict(X_test).cpu().numpy()

In [25]:

print('LinTS original mean: ', scores_more_1.mean(axis=0))
print('LinTS modified mean: ', scores_more_2.mean(axis=0))

# Check if the means are close enough
print(np.allclose(scores_more_1.mean(axis=0), scores_more_2.mean(axis=0), atol=1e-2))
print(np.allclose(scores_more_1.mean(axis=0), scores_more_2.mean(axis=0), atol=1e-2))

print("LinTS covariance: ", np.cov(scores_more_1, rowvar=False))
print("LinTS modified covariance: ", np.cov(scores_more_2, rowvar=False))
print(np.allclose(np.cov(scores_more_1, rowvar=False), np.cov(scores_more_2, rowvar=False), atol=1e-2))
print(np.allclose(np.cov(scores_more_1, rowvar=False), np.cov(scores_more_2, rowvar=False), atol=1e-2))

LinTS original mean:  [1.56298524e-04 5.00110520e-01 1.10519746e-04 5.00110520e-01]
LinTS modified mean:  [-5.18344829e-04  4.99309120e-01  1.69111780e-04  4.99577570e-01]
True
True
LinTS covariance:  [[1.00002563 0.70712491 0.70712491 0.70712491]
 [0.70712491 0.50001282 0.50001282 0.50001282]
 [0.70712491 0.50001282 0.50001282 0.50001282]
 [0.70712491 0.50001282 0.50001282 0.50001282]]
LinTS modified covariance:  [[ 9.99865659e-01 -5.71655624e-05  6.57783780e-04  4.19714414e-04]
 [-5.71655624e-05  1.99881928e+00  5.44876873e-05  4.93168290e-04]
 [ 6.57783780e-04  5.44876873e-05  2.00084172e+00 -1.57671602e-03]
 [ 4.19714414e-04  4.93168290e-04 -1.57671602e-03  2.00003048e+00]]
False
False


In [ ]:
import torch

class _LinTS_modified2(_RidgeRegression):

    def predict(self, x: np.ndarray):
        # Scale
        if self.scaler is not None:
            x = self._scale_predict_context(x)

        # Randomly sample coefficients from multivariate normal distribution
        # Covariance is enhanced with the exploration factor
        # Generates  random samples for all contexts in one single go. type(beta_sampled): np.ndarray
        eps = torch.from_numpy(self.rng.standard_normal(size=(x.shape[0], self.beta.shape[0]))).cuda()
        L = torch.linalg.cholesky(torch.from_numpy(np.square(self.alpha) * self.A_inv).cuda())
        mean = torch.from_numpy(self.beta).cuda()
        beta_sampled = mean + eps @ L.T

        # Calculate expectation y = x * beta_sampled
        return torch.sum(torch.from_numpy(x).cuda() * beta_sampled, axis=1)
    

class _LinTSOptimized2(_RidgeRegressionOptimized):

    def __init__(self, rng: _BaseRNG, alpha: Num = 1.0, l2_lambda: Num = 1.0, scale: bool = False):
        super().__init__(rng, alpha, l2_lambda, scale)
    
    def predict(self, x: np.ndarray):
        x_torch = torch.tensor(x, device='cuda', dtype=torch.double)

        num_arms, num_features = self.beta.shape
        num_contexts = x.shape[0]

        scores = torch.zeros((num_contexts, num_arms), device='cuda', dtype=torch.double)

        eps = torch.from_numpy(self.rng.standard_normal(size=(num_contexts, num_features))).cuda()

        for start in range(0, num_arms, ITEMS_PER_BATCH):
            end = min(start + ITEMS_PER_BATCH, num_arms)  

            beta_chunk = self.beta[start:end]
            A_chunk = self.A[start:end]
            A_inv_chunk = torch.linalg.inv(A_chunk)

            L_chunk = torch.linalg.cholesky((self.alpha ** 2) * A_inv_chunk)
            beta_sampled = torch.einsum('bd,add->bad', eps, L_chunk) + beta_chunk

            scores[:, start:end] = torch.einsum('bd,bad->ba', x_torch, beta_sampled)

        return scores  # shape: [B, M]

In [27]:
lints_more_arms_3 = _LinTSOptimized2(
    rng=np.random.default_rng(42),
    alpha=1.0,
    l2_lambda=1.0,
    scale=False
)

In [28]:
lints_more_arms_3.init(num_features=5, num_arms=len(arms))

lints_more_arms_3.fit(decisions, X_train, rewards)

init demorou 0.0017380714416503906 segundos
passar para cuda demorou 0.0007092952728271484 segundos
A add demorou 0.0004999637603759766 segundos
Xty demorou 0.00016927719116210938 segundos
beta demorou 0.0011057853698730469 segundos


In [29]:
scores_more_3 = lints_more_arms_3.predict(X_test).cpu().numpy()

In [30]:

print('LinTS original mean: ', scores_more_1.mean(axis=0))
print('LinTS modified mean: ', scores_more_3.mean(axis=0))

# Check if the means are close enough
print(np.allclose(scores_more_1.mean(axis=0), scores_more_3.mean(axis=0), atol=1e-2))

print("LinTS covariance: ", np.cov(scores_more_1, rowvar=False))
print("LinTS modified covariance: ", np.cov(scores_more_3, rowvar=False))
print(np.allclose(np.cov(scores_more_1, rowvar=False), np.cov(scores_more_3, rowvar=False), atol=1e-2))

LinTS original mean:  [1.56298524e-04 5.00110520e-01 1.10519746e-04 5.00110520e-01]
LinTS modified mean:  [1.56298524e-04 5.00110520e-01 1.10519746e-04 5.00110520e-01]
True
LinTS covariance:  [[1.00002563 0.70712491 0.70712491 0.70712491]
 [0.70712491 0.50001282 0.50001282 0.50001282]
 [0.70712491 0.50001282 0.50001282 0.50001282]
 [0.70712491 0.50001282 0.50001282 0.50001282]]
LinTS modified covariance:  [[1.00002563 0.70712491 0.70712491 0.70712491]
 [0.70712491 0.50001282 0.50001282 0.50001282]
 [0.70712491 0.50001282 0.50001282 0.50001282]
 [0.70712491 0.50001282 0.50001282 0.50001282]]
True


In [31]:
class _LinTS_modified2(_RidgeRegression):

    def predict(self, x: np.ndarray):
        # Scale
        if self.scaler is not None:
            x = self._scale_predict_context(x)

        # Randomly sample coefficients from multivariate normal distribution
        # Covariance is enhanced with the exploration factor
        # Generates  random samples for all contexts in one single go. type(beta_sampled): np.ndarray
        eps = torch.from_numpy(self.rng.standard_normal(size=(x.shape[0], self.beta.shape[0]))).cuda()
        L = torch.linalg.cholesky(torch.from_numpy(np.square(self.alpha) * self.A_inv).cuda())
        mean = torch.from_numpy(self.beta).cuda()
        beta_sampled = mean + eps @ L

        # Calculate expectation y = x * beta_sampled
        return torch.sum(torch.from_numpy(x).cuda() * beta_sampled, axis=1)

class _LinTSOptimized3(_RidgeRegressionOptimized):

    def __init__(self, rng: _BaseRNG, alpha: Num = 1.0, l2_lambda: Num = 1.0, scale: bool = False):
        super().__init__(rng, alpha, l2_lambda, scale)
    
    def predict(self, x: np.ndarray):
        x_torch = torch.tensor(x, device='cuda', dtype=torch.double)  # [B, D]

        num_arms, num_features = self.beta.shape
        num_contexts = x.shape[0]

        scores = torch.zeros((num_contexts, num_arms), device='cuda', dtype=torch.double)

        print(self.beta)

        for i in range(num_arms):
            eps = torch.from_numpy(self.rng.standard_normal(size=(num_contexts, num_features))).cuda()

            # Cholesky decomposition
            L = torch.linalg.cholesky((self.alpha ** 2) * torch.linalg.inv(self.A[i]))

            beta_sampled = self.beta[i] + eps @ L.T

            scores[:, i] = torch.sum(x_torch * beta_sampled, axis=1)

        return scores  # shape: [B, M]

In [32]:
lints_more_arms_4 = _LinTSOptimized3(
    rng=np.random.default_rng(42),
    alpha=1.0,
    l2_lambda=1.0,
    scale=False
)

In [33]:
lints_more_arms_4.init(num_features=5, num_arms=len(arms))

lints_more_arms_4.fit(decisions, X_train, rewards)

init demorou 0.0014488697052001953 segundos
passar para cuda demorou 0.0005772113800048828 segundos
A add demorou 0.0004382133483886719 segundos
Xty demorou 0.00015354156494140625 segundos
beta demorou 0.0009043216705322266 segundos


In [34]:
scores_more_4 = lints_more_arms_4.predict(X_test).cpu().numpy()

tensor([[0.5000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.5000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.5000],
        [0.0000, 0.0000, 0.5000, 0.0000, 0.0000]], device='cuda:0',
       dtype=torch.float64)


In [35]:

print('LinTS original mean: ', scores_more_1.mean(axis=0))
print('LinTS modified mean: ', scores_more_4.mean(axis=0))

# Check if the means are close enough
print(np.allclose(scores_more_1.mean(axis=0), scores_more_4.mean(axis=0), atol=1e-2))
print(np.allclose(scores_more_1.mean(axis=0), scores_more_4.mean(axis=0), atol=1e-2))

print("LinTS covariance: ", np.cov(scores_more_1, rowvar=False))
print("LinTS modified covariance: ", np.cov(scores_more_4, rowvar=False))
print(np.allclose(np.cov(scores_more_1, rowvar=False), np.cov(scores_more_4, rowvar=False), atol=1e-2))
print(np.allclose(np.cov(scores_more_1, rowvar=False), np.cov(scores_more_4, rowvar=False), atol=1e-2))

LinTS original mean:  [1.56298524e-04 5.00110520e-01 1.10519746e-04 5.00110520e-01]
LinTS modified mean:  [1.56298524e-04 5.00027918e-01 2.47897748e-04 4.99704299e-01]
True
True
LinTS covariance:  [[1.00002563 0.70712491 0.70712491 0.70712491]
 [0.70712491 0.50001282 0.50001282 0.50001282]
 [0.70712491 0.50001282 0.50001282 0.50001282]
 [0.70712491 0.50001282 0.50001282 0.50001282]]
LinTS modified covariance:  [[ 1.00002563e+00 -1.61562468e-04 -6.02630559e-05  1.14322518e-04]
 [-1.61562468e-04  5.00001575e-01 -2.24885063e-04  1.63212409e-04]
 [-6.02630559e-05 -2.24885063e-04  5.00162124e-01 -1.48153346e-04]
 [ 1.14322518e-04  1.63212409e-04 -1.48153346e-04  4.99540355e-01]]
False
False
